In [1]:
import pandas as pd
import numpy as np

### 시각화 라이브러리 정의
# - 파이썬에서 사용되는 기본 시각화 라이브러리
import matplotlib.pyplot as plt

# - 히트맵 라이브러리
import seaborn as sns
import geopandas as gpd
import folium
from shapely.geometry import Polygon
from shapely.geometry import Point
from folium.plugins import MarkerCluster

import os
from dask.diagnostics import ProgressBar
import dask.dataframe as dd


from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import StandardScaler

from scipy.stats import spearmanr

import dask.dataframe as dd
from sklearn.model_selection import train_test_split
from dask_ml.wrappers import Incremental
from sklearn.ensemble import RandomForestRegressor
from dask_ml.model_selection import GridSearchCV
from dask.diagnostics import ProgressBar

import pandas as pd
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import HistGradientBoostingRegressor
from xgboost import XGBRegressor
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn import set_config
import joblib  # 모델 저장용

from model_class import Models


### 한글처리
plt.rc("font", family="Malgun Gothic")

# - 마이너스 기호 깨짐 처리
plt.rcParams["axes.unicode_minus"] = False

In [2]:
wildfire = pd.read_csv("./USA data/merged_fire.csv")
wildfire.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 906078 entries, 0 to 906077
Data columns (total 29 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   latitude               906078 non-null  float64
 1   longitude              906078 non-null  float64
 2   brightness             900596 non-null  float64
 3   bright_t31             900596 non-null  float64
 4   frp                    900596 non-null  float64
 5   T2M                    900596 non-null  float64
 6   WS2M                   900596 non-null  float64
 7   RH2M                   900596 non-null  float64
 8   PRECTOTCORR            900596 non-null  float64
 9   confidence_h           900596 non-null  float64
 10  confidence_l           900596 non-null  float64
 11  confidence_n           900596 non-null  float64
 12  daynight_D             900596 non-null  float64
 13  daynight_N             900596 non-null  float64
 14  year                   900596 non-nu

In [4]:
# 데이터를 병합 하며 결측치가 생겨 결측값 제거
wildfire = wildfire.dropna()

# 독립 변수(X): float64 타입만 추출, predicted_area_km2 제외
numeric_columns = wildfire.select_dtypes(include=['float64']).columns
X = wildfire[numeric_columns.drop('predicted_area_km2')]
y = wildfire['predicted_area_km2']

# 스피어만 상관계수 계산
print("스피어만 상관관계 분석 결과 (결측값 제거 후):")
print("-" * 50)

for value in X.columns:
    statistic, pvalue = spearmanr(X[value], y)
    
    # 출력
    if pvalue < 0.05 and abs(statistic) >= 0.2:
        print(f"상관관계 계수 {statistic:.3f}, p-value {pvalue:.4f} : ({value})는 predicted_area_km2에 대해 유의미하다 (강한 상관관계)")
    elif pvalue < 0.05:
        print(f"상관관계 계수 {statistic:.3f}, p-value {pvalue:.4f} : ({value})는 predicted_area_km2에 대해 약한 상관관계 (유의미)")
    else:
        print(f"상관관계 계수 {statistic:.3f}, p-value {pvalue:.4f} : ({value})는 predicted_area_km2에 대해 유의미하지 않다")

print("-" * 50)

스피어만 상관관계 분석 결과 (결측값 제거 후):
--------------------------------------------------
상관관계 계수 0.064, p-value 0.0000 : (latitude)는 predicted_area_km2에 대해 약한 상관관계 (유의미)
상관관계 계수 -0.011, p-value 0.0000 : (longitude)는 predicted_area_km2에 대해 약한 상관관계 (유의미)
상관관계 계수 0.813, p-value 0.0000 : (brightness)는 predicted_area_km2에 대해 유의미하다 (강한 상관관계)
상관관계 계수 0.640, p-value 0.0000 : (bright_t31)는 predicted_area_km2에 대해 유의미하다 (강한 상관관계)
상관관계 계수 1.000, p-value 0.0000 : (frp)는 predicted_area_km2에 대해 유의미하다 (강한 상관관계)
상관관계 계수 0.154, p-value 0.0000 : (T2M)는 predicted_area_km2에 대해 약한 상관관계 (유의미)
상관관계 계수 -0.026, p-value 0.0000 : (WS2M)는 predicted_area_km2에 대해 약한 상관관계 (유의미)
상관관계 계수 -0.198, p-value 0.0000 : (RH2M)는 predicted_area_km2에 대해 약한 상관관계 (유의미)
상관관계 계수 -0.073, p-value 0.0000 : (PRECTOTCORR)는 predicted_area_km2에 대해 약한 상관관계 (유의미)
상관관계 계수 0.339, p-value 0.0000 : (confidence_h)는 predicted_area_km2에 대해 유의미하다 (강한 상관관계)
상관관계 계수 0.202, p-value 0.0000 : (confidence_l)는 predicted_area_km2에 대해 유의미하다 (강한 상관관계)
상관관계 계수 -0.404, p-

In [3]:
train_cols = [
    "frp",           # 강한 상관관계
    "brightness",    # 강한 상관관계
    "bright_t31",    # 강한 상관관계
    "confidence_h",  # 강한 상관관계
    "confidence_l",  # 강한 상관관계
    "confidence_n",  # 강한 상관관계
    "daynight_D",    # 강한 상관관계
    "daynight_N",    # 강한 상관관계
    "latitude",      # 약한 상관관계
    "longitude",     # 약한 상관관계
    "T2M",           # 약한 상관관계
    "WS2M",          # 약한 상관관계
    "RH2M",          # 약한 상관관계
    "PRECTOTCORR",   # 약한 상관관계
    "year",          # 약한 상관관계
    "month",         # 약한 상관관계
    "day",           # 약한 상관관계
    "season",        # 약한 상관관계
    "weekday",       # 약한 상관관계
    "WS2M_RH2M_interaction",  # 약한 상관관계
    "high_temperature",       # 약한 상관관계
    "precipitation_flag"      # 약한 상관관계
]

# 데이터 준비
train = wildfire[train_cols]
target = wildfire["predicted_area_km2"]

# 첫 번째 분할 (훈련 데이터와 검증 데이터)
train_input, val_input, train_target, val_target = train_test_split(train, target, test_size=0.4, random_state=42, shuffle=True)

# 두 번째 분할 (검증 데이터와 테스트 데이터)
val_input, test_input, val_target, test_target = train_test_split(val_input, val_target, test_size=0.5, random_state=42, shuffle=True)

print(train_input.shape, train_target.shape)
print(val_input.shape, val_target.shape)
print(test_input.shape, test_target.shape)


(543646, 22) (543646,)
(181216, 22) (181216,)
(181216, 22) (181216,)


In [ ]:
rf = RandomForestRegressor() 

rf.fit(train_input, train_target)

train_score = rf.score(train_input, train_target)
val_score = rf.score(val_input, val_target)
test_score = rf.score(test_input, test_target)

print(f"train_score: {train_score}, val_score: {val_score}, test_score: {test_score}, 과적합여부: {train_score - val_score}")

train_score: 0.9999999997156066, val_score: 0.9999999978892833, test_score: 0.9999999981095657, 과적합여부: 1.8263233148019253e-09


In [ ]:
models = Models()
models.total_models(train_input, train_target, val_input, val_target, test_input, test_target)

Tuning and training RandomForest with None...
Best Parameters for RandomForest: {'n_estimators': 200, 'min_samples_split': 10, 'max_depth': 20}
-*** RandomForest with None ***-
훈련: 0.9982, 검증: 0.9948, 테스트: 0.9955, 과적합여부: 0.0034
사용 가능한 모델입니다 (일반화).

Saved RandomForest with None as final model.
Tuning and training HistGradientBoosting with None...
Best Parameters for HistGradientBoosting: {'max_iter': 200, 'max_depth': 5, 'learning_rate': 0.1}
-*** HistGradientBoosting with None ***-
훈련: 0.9947, 검증: 0.9939, 테스트: 0.9947, 과적합여부: 0.0008
사용 가능한 모델입니다 (일반화).

Saved HistGradientBoosting with None as final model.
Tuning and training XGB with None...
Best Parameters for XGB: {'subsample': 1.0, 'n_estimators': 100, 'max_depth': 6, 'learning_rate': 0.1}
-*** XGB with None ***-
훈련: 0.9948, 검증: 0.9940, 테스트: 0.9947, 과적합여부: 0.0008
사용 가능한 모델입니다 (일반화).

Saved XGB with None as final model.
Tuning and training RandomForest with Standard...


In [ ]:
results_df = models.get_results()
print(results_df)
results_df.to_csv("model_results_final.csv", index=False)

In [ ]:
# 모델 실행
models = Models()
models.total_models(train_input, train_target, val_input, val_target, test_input, test_target)
results_df = models.get_results()
print(results_df)
results_df.to_csv("model_results_final.csv", index=False)

In [ ]:

# 모델 실행
models = Models()
models.total_models(train_input, train_target, val_input, val_target, test_input, test_target)
results_df = models.get_results()
print(results_df)
results_df.to_csv("model_results_final.csv", index=False)